# Automatically detecting if this is a remote load, rather than needing to include it each time

In [13]:
%load_ext autoreload
%autoreload 2 
import sys
sys.path.append("../../../xapres/")
import xapres as xa
import numpy as np
import xarray as xr
import os
import gcsfs

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
fd = xa.load.from_dats(loglevel = 'DEBUG')
root_directory = 'gs://ldeo-glaciology/apres/thwaites'
filenames = fd.list_files(directory = root_directory)
directory = os.path.dirname(filenames[20])
ds = fd.load_all(directory = directory, file_numbers_to_process=[0]) 

DEBUG    16:54:35 	 load.py @function _setup_logging line 770 - Stream logging level set to DEBUG
DEBUG    16:54:35 	 load.py @function _setup_logging line 771 - Add console handler to logger
DEBUG    16:54:35 	 load.py @function _setup_logging line 784 - File logging level set to DEBUG
DEBUG    16:54:35 	 load.py @function list_files line 135 - Find all the dat files in the directory gs://ldeo-glaciology/apres/thwaites
DEBUG    16:54:39 	 load.py @function is_this_a_remote_load line 793 - remote_load set to True
DEBUG    16:54:41 	 load.py @function list_files line 154 - Finish call to list_files. Found 2301 files
DEBUG    16:54:45 	 load.py @function is_this_a_remote_load line 793 - remote_load set to True
DEBUG    16:54:45 	 load.py @function load_all line 259 - Start call to load_all with remote_load = True, directory = gs://ldeo-glaciology/apres/thwaites/2022-2023/Point/G1-25-05, file_numbers_to_process = [0], file_names_to_process = None, bursts_to_process = All, attended = False

In [31]:
ds

<xarray.Dataset> Size: 97MB
Dimensions:          (time: 1, chirp_time: 40001, chirp_num: 100,
                      attenuator_setting_pair: 1, profile_range: 39999)
Coordinates:
  * time             (time) datetime64[ns] 8B 2022-12-31T09:18:13
  * chirp_time       (chirp_time) float64 320kB 0.0 2.5e-05 5e-05 ... 1.0 1.0
  * chirp_num        (chirp_num) int64 800B 0 1 2 3 4 5 6 ... 94 95 96 97 98 99
    filename         (time) <U91 364B 'gs://ldeo-glaciology/apres/thwaites/20...
    burst_number     (time) int64 8B 0
    AFGain           (attenuator_setting_pair) int64 8B -4
    attenuator       (attenuator_setting_pair) float64 8B 26.0
    orientation      (time) <U7 28B 'unknown'
  * profile_range    (profile_range) float64 320kB 0.0 0.2103 ... 8.411e+03
Dimensions without coordinates: attenuator_setting_pair
Data variables:
    chirp            (time, chirp_time, chirp_num, attenuator_setting_pair) float64 32MB ...
    latitude         (time) float64 8B 0.0
    longitude        (time) float64 8B 0.0
    battery_voltage  (time) float64 8B 0.2296
    temperature_1    (time) float64 8B -6.523
    temperature_2    (time) float64 8B -7.047
    profile          (time, chirp_num, attenuator_setting_pair, profile_range) complex128 64MB ...
Attributes:
    constants:   {'c': 300000000.0, 'K': 200000000.0, 'f_1': 200000000, 'f_2'...
    processing:  Created on 2024-10-14 16:49:41.776642

With the changes I was making to how remote_load is passed around inside the load calss, I found a bug which resulted in remote_load starting out as positive (because is_this_a_remote_load found 'gs://') and then being set to False (because list_files was not supplying the fullnames with the 'gs://').
This is how list_files worked before. It missed out the 'gs://' part of the filename.:

In [35]:
fs = gcsfs.GCSFileSystem()
dat_filenames = fs.glob(root_directory + '/**/*' + '.[dD][aA][tT]', recursive = True)
dat_filenames

['ldeo-glaciology/apres/thwaites/2022-2023/EAGERER/GA04/20221207_TEST/._Survey_2022-12-07_050827.dat',
 'ldeo-glaciology/apres/thwaites/2022-2023/EAGERER/GA04/20221207_TEST/Survey_2022-12-07_050827.dat',
 'ldeo-glaciology/apres/thwaites/2022-2023/EAGERER/GA07/20221206_TEST/._Survey_2022-12-06_042707.dat',
 'ldeo-glaciology/apres/thwaites/2022-2023/EAGERER/GA07/20221206_TEST/Survey_2022-12-06_042707.dat',
 'ldeo-glaciology/apres/thwaites/2022-2023/EAGERER/GA10/20221206_Test/._Survey_2022-12-06_024904.dat',
 'ldeo-glaciology/apres/thwaites/2022-2023/EAGERER/GA10/20221206_Test/Survey_2022-12-06_024904.dat',
 'ldeo-glaciology/apres/thwaites/2022-2023/EAGERER/GA10/G1-20221218/DIR2022-12-06-0312/DATA2022-12-06-0312.DAT',
 'ldeo-glaciology/apres/thwaites/2022-2023/EAGERER/GA10/G1-20221218/DIR2022-12-06-0312/DATA2022-12-06-0342.DAT',
 'ldeo-glaciology/apres/thwaites/2022-2023/EAGERER/GA10/G1-20221218/DIR2022-12-06-0312/DATA2022-12-06-0412.DAT',
 'ldeo-glaciology/apres/thwaites/2022-2023/EAGERE

I have added the following to load.list_files to add the gs:// back in:

In [36]:
full_filenames = ['gs://' + dat_filename for dat_filename in dat_filenames]
directory = os.path.dirname(full_filenames[20])
directory


'gs://ldeo-glaciology/apres/thwaites/2022-2023/Point/G1-25-05'

In [25]:
full_filenames

['gs://ldeo-glaciology/apres/thwaites/2022-2023/EAGERER/GA04/20221207_TEST/._Survey_2022-12-07_050827.dat',
 'gs://ldeo-glaciology/apres/thwaites/2022-2023/EAGERER/GA04/20221207_TEST/Survey_2022-12-07_050827.dat',
 'gs://ldeo-glaciology/apres/thwaites/2022-2023/EAGERER/GA07/20221206_TEST/._Survey_2022-12-06_042707.dat',
 'gs://ldeo-glaciology/apres/thwaites/2022-2023/EAGERER/GA07/20221206_TEST/Survey_2022-12-06_042707.dat',
 'gs://ldeo-glaciology/apres/thwaites/2022-2023/EAGERER/GA10/20221206_Test/._Survey_2022-12-06_024904.dat',
 'gs://ldeo-glaciology/apres/thwaites/2022-2023/EAGERER/GA10/20221206_Test/Survey_2022-12-06_024904.dat',
 'gs://ldeo-glaciology/apres/thwaites/2022-2023/EAGERER/GA10/G1-20221218/DIR2022-12-06-0312/DATA2022-12-06-0312.DAT',
 'gs://ldeo-glaciology/apres/thwaites/2022-2023/EAGERER/GA10/G1-20221218/DIR2022-12-06-0312/DATA2022-12-06-0342.DAT',
 'gs://ldeo-glaciology/apres/thwaites/2022-2023/EAGERER/GA10/G1-20221218/DIR2022-12-06-0312/DATA2022-12-06-0412.DAT',
 'gs

load_all now works without needing to specify remote_load=True. But if you do it doesnt return an error. 

In [38]:
fd = xa.load.from_dats()
ds = fd.load_all(directory = directory, file_numbers_to_process=[0]) 
ds

<xarray.Dataset> Size: 97MB
Dimensions:          (time: 1, chirp_time: 40001, chirp_num: 100,
                      attenuator_setting_pair: 1, profile_range: 39999)
Coordinates:
  * time             (time) datetime64[ns] 8B 2022-12-31T09:18:13
  * chirp_time       (chirp_time) float64 320kB 0.0 2.5e-05 5e-05 ... 1.0 1.0
  * chirp_num        (chirp_num) int64 800B 0 1 2 3 4 5 6 ... 94 95 96 97 98 99
    filename         (time) <U91 364B 'gs://ldeo-glaciology/apres/thwaites/20...
    burst_number     (time) int64 8B 0
    AFGain           (attenuator_setting_pair) int64 8B -4
    attenuator       (attenuator_setting_pair) float64 8B 26.0
    orientation      (time) <U7 28B 'unknown'
  * profile_range    (profile_range) float64 320kB 0.0 0.2103 ... 8.411e+03
Dimensions without coordinates: attenuator_setting_pair
Data variables:
    chirp            (time, chirp_time, chirp_num, attenuator_setting_pair) float64 32MB ...
    latitude         (time) float64 8B 0.0
    longitude        (time) float64 8B 0.0
    battery_voltage  (time) float64 8B 0.2296
    temperature_1    (time) float64 8B -6.523
    temperature_2    (time) float64 8B -7.047
    profile          (time, chirp_num, attenuator_setting_pair, profile_range) complex128 64MB ...
Attributes:
    constants:   {'c': 300000000.0, 'K': 200000000.0, 'f_1': 200000000, 'f_2'...
    processing:  Created on 2024-10-14 17:08:58.478535

In [40]:
ds = fd.load_all(directory = directory, file_numbers_to_process=[0], remote_load = True) 
ds

<xarray.Dataset> Size: 97MB
Dimensions:          (time: 1, chirp_time: 40001, chirp_num: 100,
                      attenuator_setting_pair: 1, profile_range: 39999)
Coordinates:
  * time             (time) datetime64[ns] 8B 2022-12-31T09:18:13
  * chirp_time       (chirp_time) float64 320kB 0.0 2.5e-05 5e-05 ... 1.0 1.0
  * chirp_num        (chirp_num) int64 800B 0 1 2 3 4 5 6 ... 94 95 96 97 98 99
    filename         (time) <U91 364B 'gs://ldeo-glaciology/apres/thwaites/20...
    burst_number     (time) int64 8B 0
    AFGain           (attenuator_setting_pair) int64 8B -4
    attenuator       (attenuator_setting_pair) float64 8B 26.0
    orientation      (time) <U7 28B 'unknown'
  * profile_range    (profile_range) float64 320kB 0.0 0.2103 ... 8.411e+03
Dimensions without coordinates: attenuator_setting_pair
Data variables:
    chirp            (time, chirp_time, chirp_num, attenuator_setting_pair) float64 32MB ...
    latitude         (time) float64 8B 0.0
    longitude        (time) float64 8B 0.0
    battery_voltage  (time) float64 8B 0.2296
    temperature_1    (time) float64 8B -6.523
    temperature_2    (time) float64 8B -7.047
    profile          (time, chirp_num, attenuator_setting_pair, profile_range) complex128 64MB ...
Attributes:
    constants:   {'c': 300000000.0, 'K': 200000000.0, 'f_1': 200000000, 'f_2'...
    processing:  Created on 2024-10-14 17:09:30.590186